In [1]:
# Instalação de pacotes e importação das bibliotecas que serão utilizadas no projeto
!python -m pip install "dask[dataframe]"

import pandas as pd
import numpy as np
import dask.dataframe as dd

import shutil

You should consider upgrading via the '/home/user/anaconda3/bin/python -m pip install --upgrade pip' command.


/home/user/anaconda3/lib/python3.7/site-packages/dask/dataframe/utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
# filtragem das variáveis com as quais quero trabalhar
cols = ['CBO Ocupação 2002', 'Faixa Etária', 'Município', 
        'Vl Remun Média Nom', 'Sexo Trabalhador']

In [3]:
# leitura dos arquivos em um dataframe dask
df = dd.read_csv('rais_raw_data/RAIS_VINC*',sep = ";", encoding= "ISO-8859-1", usecols = cols, low_memory=False, 
                 dtype={'CBO Ocupação 2002': 'object','Faixa Etária': 'object'})

In [4]:
# remoção de linhas que não interessam ao projeto e consequente redução do dataset
df = df[df['CBO Ocupação 2002'].str.startswith('2')]

In [5]:
# transformação de um dask dataframe para um pandas dataframe
df = df.compute()

In [6]:
# verificação do número de linhas e colunas do dataframe
df.shape

(7072962, 5)

In [7]:
df.head()

,CBO Ocupação 2002,Faixa Etária,Município,Vl Remun Média Nom,Sexo Trabalhador
380,231305,05,500240,"0000003591,52",1
758,231305,05,500240,"0000003750,03",1
770,252210,05,500580,"0000000000,00",1
771,252210,05,500580,"0000013500,00",1
781,224125,03,500210,"0000001224,28",1


In [8]:
# carregando os dados complementares do projeto em dataframes pandas
cbo = pd.read_csv('supplementary_data/cbo.csv')
estado = pd.read_csv('supplementary_data/estado.csv')

In [9]:
cbo.head()

,CBO Ocupação 2002,Profissão
0,251505,Psicólogo(a)
1,251510,Psicólogo(a)
2,251515,Psicólogo(a)
3,251520,Psicólogo(a)
4,251525,Psicólogo(a)


In [10]:
estado.head()

,Município,Estado
0,110001,Rondônia
1,110002,Rondônia
2,110003,Rondônia
3,110004,Rondônia
4,110005,Rondônia


In [11]:
# junção - join - do dataframe original com o dataframe com os nomes das profissões
data = pd.merge(df, cbo, on='CBO Ocupação 2002', how ='left')

In [12]:
data.head()

,CBO Ocupação 2002,Faixa Etária,Município,Vl Remun Média Nom,Sexo Trabalhador,Profissão
0,231305,05,500240,"0000003591,52",1,NaN
1,231305,05,500240,"0000003750,03",1,NaN
2,252210,05,500580,"0000000000,00",1,NaN
3,252210,05,500580,"0000013500,00",1,NaN
4,224125,03,500210,"0000001224,28",1,NaN


In [13]:
# remoção das linhas sem o nome de alguma profissão
data = data[data['Profissão'].notnull()]

In [14]:
data.shape

(1830924, 6)

In [15]:
# junção - join - do dataframe anterior com o dataframe com os estados
data = pd.merge(data,estado,on ='Município', how ='left')

In [16]:
data.head()

,CBO Ocupação 2002,Faixa Etária,Município,Vl Remun Média Nom,Sexo Trabalhador,Profissão,Estado
0,251225,06,500110,"0000002687,50",1,Economista,Mato Grosso do Sul
1,212420,05,500270,"0000002974,42",1,Analista de sistemas,Mato Grosso do Sul
2,212420,05,500270,"0000003591,73",1,Analista de sistemas,Mato Grosso do Sul
3,212420,05,500570,"0000000000,00",1,Analista de sistemas,Mato Grosso do Sul
4,212420,05,500620,"0000001200,00",1,Analista de sistemas,Mato Grosso do Sul


In [17]:
# remoção das colunas com o código do CBO e com o município
data.drop(['CBO Ocupação 2002','Município'], axis=1, inplace=True)

In [18]:
data['Vl Remun Média Nom']=data['Vl Remun Média Nom'].str.replace(',','.')

In [19]:
# transformação das variáveis com os valores dos salários de "object" para "float64"
data["Vl Remun Média Nom"] = pd.to_numeric(data["Vl Remun Média Nom"])

In [20]:
data.dtypes

Faixa Etária           object
Vl Remun Média Nom    float64
Sexo Trabalhador        int64
Profissão              object
Estado                 object
dtype: object

In [21]:
# remoção das linhas em que os valores de salário são iguais a 0
data = data[data['Vl Remun Média Nom'] > 0]

In [22]:
data.shape

(1763221, 5)

In [23]:
# troca da codificação da coluna "Faixa Etária" pelo valor de fato da variável "Faixa Etária"
data['Faixa Etária'] = data['Faixa Etária'].map({'01': '10 a 14 anos',
                                               '02': '15 a 17 anos',
                                               '03': '18 a 24 anos',
                                               '04': '25 a 29 anos',
                                               '05': '30 a 39 anos',
                                               '06': '40 a 49 anos',
                                               '07': '50 a 64 anos',
                                               '08': '65 anos ou mais'
                                              }
                                             )

In [24]:
# troca da codificação da coluna "Sexo Trabalhador" pelo valor de fato da variável "Sexo Trabalhador"
data['Sexo Trabalhador'] = data['Sexo Trabalhador'].map({1: 'Masculino',
                                                         2: 'Feminino',
                                                        -1: 'sem informacao'
                                                        }
                                                       )

In [25]:
# renomeação dos nomes das colunas
data.rename(columns={'Vl Remun Média Nom' : 'Remuneração (em R$)',
                     'Sexo Trabalhador': 'Gênero', 
                     }, 
            inplace=True)

In [26]:
data.head()

,Faixa Etária,Remuneração (em R$),Gênero,Profissão,Estado
0,40 a 49 anos,2687.50,Masculino,Economista,Mato Grosso do Sul
1,30 a 39 anos,2974.42,Masculino,Analista de sistemas,Mato Grosso do Sul
2,30 a 39 anos,3591.73,Masculino,Analista de sistemas,Mato Grosso do Sul
4,30 a 39 anos,1200.00,Masculino,Analista de sistemas,Mato Grosso do Sul
5,18 a 24 anos,1200.00,Masculino,Analista de sistemas,Mato Grosso do Sul


In [27]:
# distribuição dos valores da variável "Faixa Etária"
data['Faixa Etária'].value_counts()

30 a 39 anos       720491
40 a 49 anos       381844
25 a 29 anos       282559
50 a 64 anos       251748
18 a 24 anos        78348
65 anos ou mais     47966
15 a 17 anos          258
10 a 14 anos            4
Name: Faixa Etária, dtype: int64

In [28]:
# remoção de linhas com preenchimento provavelmente incorreto
data.drop(data.loc[data['Faixa Etária']=='15 a 17 anos'].index, inplace=True)
data.drop(data.loc[data['Faixa Etária']=='10 a 14 anos'].index, inplace=True)

In [29]:
# distribuição dos valores da variável "Gênero"
data['Gênero'].value_counts()

Feminino     908790
Masculino    854169
Name: Gênero, dtype: int64

In [30]:
# distribuição dos valores da variável "Profissão"
data['Profissão'].value_counts()

Analista de sistemas        414797
Enfermeiro(a)               355883
Medico(a)                   316154
Engenheiro(a)               209871
Advogado(a)                  86253
Psicólogo(a)                 77794
Dentista                     75335
Fisioterapeuta               72932
Nutricionista                59832
Economista                   52790
Medico(a) veterinario(a)     23757
Arquiteto(a)                 17561
Name: Profissão, dtype: int64

In [31]:
# distribuição dos valores da variável "Estado"
data['Estado'].value_counts()

São Paulo              597647
Rio de Janeiro         192102
Minas Gerais           187255
Paraná                  96335
Rio Grande do Sul       94523
Santa Catarina          71937
Bahia                   71249
Distrito Federal        61752
Pernambuco              60591
Ceará                   46316
Espírito Santo          33608
Goiás                   32048
Pará                    30752
Paraíba                 26077
Mato Grosso do Sul      20923
Rio Grande do Norte     19413
Mato Grosso             18955
Amazonas                17956
Maranhão                17247
Sergipe                 14612
Piauí                   14459
Tocantins               12175
Alagoas                 11283
Rondônia                 6825
Acre                     2518
Amapá                    2408
Roraima                  1993
Name: Estado, dtype: int64

In [32]:
# remoção das linhas em que os valores de salário são superiores a 30 mil
data = data[data['Remuneração (em R$)'] < 30000]

In [33]:
salary_df = data[['Profissão', 'Remuneração (em R$)']]

In [34]:
age_df = data[['Profissão', 'Faixa Etária']]

In [35]:
gender_df = data[['Profissão', 'Gênero']]

In [36]:
local_df = data[['Profissão','Estado']]

In [37]:
# criação do dataset com dados de salário para desenvolvimento do data app
salary_df.to_csv('salary.csv', index=False)

In [38]:
# criação do dataset com dados de idade para desenvolvimento do data app
age_df.to_csv('age.csv', index=False)

In [39]:
# criação do dataset com dados de gênero para desenvolvimento do data app
gender_df.to_csv('gender.csv', index=False)

In [40]:
# criação do dataset com dados de localização para desenvolvimento do data app
local_df.to_csv('local.csv', index=False)

In [41]:
# comando para mover o arquivo "dataset.csv" para o diretório "app_dataset"
shutil.move("salary.csv", "app_dataset/salary.csv")
shutil.move("age.csv", "app_dataset/age.csv")
shutil.move("gender.csv", "app_dataset/gender.csv")
shutil.move("local.csv", "app_dataset/local.csv")

'app_dataset/local.csv'